## 📌 Attribution
This notebook is a modified and extended version of public work originally shared on Kaggle by [@taylorsamarel](https://www.kaggle.com/taylorsamarel).

- 📘 [REBOOT: FIRE AND ICE](https://www.kaggle.com/code/taylorsamarel/reboot-fire-and-ice)

This notebook is intended solely for educational and research purposes. All original rights remain with the original author.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

from xgboost import XGBRegressor
import xgboost as xgb
from sklearn.model_selection import KFold, TimeSeriesSplit
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.feature_selection import SelectKBest, mutual_info_regression
from scipy.stats import pearsonr, rankdata, spearmanr
from scipy.optimize import minimize, differential_evolution
import gc
from collections import defaultdict
import matplotlib.pyplot as plt
import seaborn as sns

print("Starting Bootstrap Stability Analysis for XGBoost Pipeline...")
print("=" * 80)

# Configuration
class CFG:
    train_path = "/kaggle/input/drw-crypto-market-prediction/train.parquet"
    test_path = "/kaggle/input/drw-crypto-market-prediction/test.parquet"
    sample_sub_path = "/kaggle/input/drw-crypto-market-prediction/sample_submission.csv"
    
    # Bootstrap settings
    n_bootstraps = 10  # Number of bootstrap iterations
    bootstrap_sample_ratio = 0.8  # Proportion of data to sample
    cv_folds = 5  # Cross-validation folds per bootstrap
    
    # Stability thresholds
    feature_stability_threshold = 0.7  # Feature must appear in 70% of bootstraps
    score_stability_threshold = 0.05  # Max acceptable std dev in CV scores
    
    random_state = 42
    use_gpu = False
    
    # Feature settings - balanced approach
    max_x_features = 80
    n_interaction_features = 50
    n_proprietary_features = 30
    
    # Feature selection
    use_feature_selection = True
    feature_selection_threshold = 0.01

# Bootstrap stability tracker
class BootstrapStabilityTracker:
    def __init__(self):
        self.feature_importance_records = []
        self.cv_scores = defaultdict(list)
        self.train_scores = defaultdict(list)
        self.feature_selection_counts = defaultdict(int)
        self.model_predictions = defaultdict(list)
        self.bootstrap_indices = []
        
    def record_iteration(self, iteration, features, importance_df, cv_scores, train_scores, predictions):
        """Record results from one bootstrap iteration"""
        self.feature_importance_records.append({
            'iteration': iteration,
            'features': features,
            'importance_df': importance_df
        })
        
        for feat in features:
            self.feature_selection_counts[feat] += 1
            
        for model_name, score in cv_scores.items():
            self.cv_scores[model_name].append(score)
            
        for model_name, score in train_scores.items():
            self.train_scores[model_name].append(score)
            
        for model_name, pred in predictions.items():
            self.model_predictions[model_name].append(pred)
    
    def get_stable_features(self, min_appearance_ratio=0.7):
        """Get features that appear in at least min_appearance_ratio of bootstraps"""
        total_iterations = len(self.feature_importance_records)
        stable_features = []
        
        for feat, count in self.feature_selection_counts.items():
            if count / total_iterations >= min_appearance_ratio:
                stable_features.append((feat, count / total_iterations))
                
        return sorted(stable_features, key=lambda x: x[1], reverse=True)
    
    def get_model_stability_metrics(self):
        """Calculate stability metrics for each model"""
        stability_metrics = {}
        
        for model_name in self.cv_scores:
            cv_scores_array = np.array(self.cv_scores[model_name])
            train_scores_array = np.array(self.train_scores[model_name])
            
            stability_metrics[model_name] = {
                'cv_mean': np.mean(cv_scores_array),
                'cv_std': np.std(cv_scores_array),
                'cv_min': np.min(cv_scores_array),
                'cv_max': np.max(cv_scores_array),
                'train_mean': np.mean(train_scores_array),
                'train_std': np.std(train_scores_array),
                'overfit_score': np.mean(train_scores_array) - np.mean(cv_scores_array)
            }
            
        return stability_metrics
    
    def plot_stability_analysis(self):
        """Create visualization of stability analysis"""
        fig, axes = plt.subplots(2, 2, figsize=(15, 12))
        
        # Plot 1: Feature appearance frequency
        stable_features = self.get_stable_features(0.5)[:20]
        features, frequencies = zip(*stable_features)
        
        axes[0, 0].barh(features, frequencies)
        axes[0, 0].set_xlabel('Appearance Frequency')
        axes[0, 0].set_title('Top 20 Most Stable Features')
        axes[0, 0].axvline(x=0.7, color='r', linestyle='--', label='Stability Threshold')
        
        # Plot 2: Model CV score distributions
        model_names = list(self.cv_scores.keys())
        cv_data = [self.cv_scores[name] for name in model_names]
        
        axes[0, 1].boxplot(cv_data, labels=model_names)
        axes[0, 1].set_ylabel('CV Score')
        axes[0, 1].set_title('Model CV Score Distributions')
        axes[0, 1].tick_params(axis='x', rotation=45)
        
        # Plot 3: Train vs CV scores
        stability_metrics = self.get_model_stability_metrics()
        
        for model_name in model_names:
            metrics = stability_metrics[model_name]
            axes[1, 0].scatter(metrics['cv_mean'], metrics['train_mean'], 
                             s=100, label=model_name, alpha=0.7)
            axes[1, 0].errorbar(metrics['cv_mean'], metrics['train_mean'],
                              xerr=metrics['cv_std'], yerr=metrics['train_std'],
                              alpha=0.3)
        
        # Add diagonal line
        min_val = min(axes[1, 0].get_xlim()[0], axes[1, 0].get_ylim()[0])
        max_val = max(axes[1, 0].get_xlim()[1], axes[1, 0].get_ylim()[1])
        axes[1, 0].plot([min_val, max_val], [min_val, max_val], 'k--', alpha=0.3)
        
        axes[1, 0].set_xlabel('Mean CV Score')
        axes[1, 0].set_ylabel('Mean Train Score')
        axes[1, 0].set_title('Train vs CV Performance')
        axes[1, 0].legend()
        
        # Plot 4: Overfitting scores
        overfit_scores = [(name, metrics['overfit_score']) 
                         for name, metrics in stability_metrics.items()]
        overfit_scores.sort(key=lambda x: x[1])
        
        names, scores = zip(*overfit_scores)
        colors = ['green' if s < 0.1 else 'orange' if s < 0.2 else 'red' for s in scores]
        
        axes[1, 1].barh(names, scores, color=colors)
        axes[1, 1].set_xlabel('Overfitting Score (Train - CV)')
        axes[1, 1].set_title('Model Overfitting Analysis')
        axes[1, 1].axvline(x=0.1, color='g', linestyle='--', alpha=0.5)
        axes[1, 1].axvline(x=0.2, color='r', linestyle='--', alpha=0.5)
        
        plt.tight_layout()
        plt.savefig('bootstrap_stability_analysis.png', dpi=300)
        plt.show()

# Memory optimization (keep from original)
def reduce_mem_usage(df, name=""):
    print(f"Optimizing memory for {name}...")
    start_mem = df.memory_usage().sum() / 1024**2
    
    for col in df.columns:
        col_type = df[col].dtype
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
    
    end_mem = df.memory_usage().sum() / 1024**2
    print(f'Memory usage: {start_mem:.2f} MB -> {end_mem:.2f} MB ({100*(start_mem-end_mem)/start_mem:.1f}% reduction)')
    return df

# Create bootstrap sample
def create_bootstrap_sample(df, sample_ratio=0.8, random_state=None):
    """Create a bootstrap sample of the dataframe"""
    n_samples = int(len(df) * sample_ratio)
    indices = np.random.RandomState(random_state).choice(len(df), n_samples, replace=True)
    return df.iloc[indices].reset_index(drop=True), indices

# Cross-validation with stability tracking
def stable_cross_validation(X, y, model_params, n_folds=5, random_state=42):
    """Perform cross-validation and return both mean score and fold scores"""
    kf = KFold(n_splits=n_folds, shuffle=True, random_state=random_state)
    fold_scores = []
    
    for fold, (train_idx, valid_idx) in enumerate(kf.split(X)):
        X_fold_train = X.iloc[train_idx] if hasattr(X, 'iloc') else X[train_idx]
        y_fold_train = y.iloc[train_idx] if hasattr(y, 'iloc') else y[train_idx]
        X_fold_valid = X.iloc[valid_idx] if hasattr(X, 'iloc') else X[valid_idx]
        y_fold_valid = y.iloc[valid_idx] if hasattr(y, 'iloc') else y[valid_idx]
        
        model = XGBRegressor(**model_params)
        model.fit(X_fold_train, y_fold_train)
        
        pred_valid = model.predict(X_fold_valid)
        fold_score = pearsonr(y_fold_valid, pred_valid)[0]
        fold_scores.append(fold_score)
    
    return np.mean(fold_scores), np.std(fold_scores), fold_scores

# Feature importance with stability
def get_stable_feature_importance(X, y, feature_names, n_runs=5):
    """Get feature importance with stability across multiple runs"""
    importance_runs = []
    
    for i in range(n_runs):
        params = {
            'n_estimators': 100,
            'max_depth': 6,
            'learning_rate': 0.1,
            'subsample': 0.8,
            'colsample_bytree': 0.8,
            'random_state': 42 + i,
            'n_jobs': -1,
            'verbosity': 0
        }
        
        model = XGBRegressor(**params)
        model.fit(X, y)
        importance_runs.append(model.feature_importances_)
    
    # Calculate mean and std of importance
    importance_mean = np.mean(importance_runs, axis=0)
    importance_std = np.std(importance_runs, axis=0)
    
    # Create DataFrame with stability metrics
    importance_df = pd.DataFrame({
        'feature': feature_names,
        'importance_mean': importance_mean,
        'importance_std': importance_std,
        'importance_cv': importance_std / (importance_mean + 1e-8),
        'stability_score': importance_mean / (importance_std + 1e-8)
    }).sort_values('importance_mean', ascending=False)
    
    return importance_df

# Modified feature engineering functions (keep from original but add stability tracking)
def create_proprietary_x_variables(df, n_features=30):
    """Create proprietary X variables with focus on quality over quantity"""
    print(f"Creating {n_features} proprietary X variables...")
    
    # Get existing X features
    x_features = [col for col in df.columns if col.startswith('X') and col[1:].isdigit()]
    
    # Important X features from original pipeline
    important_x = ["X752", "X287", "X298", "X759", "X302", "X55", "X56", "X52", "X303", "X51",
                   "X344", "X598", "X385", "X603", "X674", "X415", "X345", "X137", "X174", "X178"]
    
    # Use available important features
    base_features = [f for f in important_x if f in df.columns][:10]
    
    # Add some high-variance features
    if len(base_features) < 10:
        x_variances = df[x_features].var()
        high_var_features = x_variances.nlargest(10).index.tolist()
        for feat in high_var_features:
            if feat not in base_features:
                base_features.append(feat)
                if len(base_features) >= 10:
                    break
    
    prop_idx = 1
    
    # 1. Statistical combinations (8 features)
    if len(base_features) >= 5:
        df[f'X_prop_{prop_idx}'] = df[base_features[:5]].mean(axis=1)
        prop_idx += 1
        df[f'X_prop_{prop_idx}'] = df[base_features[:5]].std(axis=1)
        prop_idx += 1
        df[f'X_prop_{prop_idx}'] = df[base_features[:5]].max(axis=1) - df[base_features[:5]].min(axis=1)
        prop_idx += 1
        df[f'X_prop_{prop_idx}'] = df[base_features[:5]].median(axis=1)
        prop_idx += 1
        df[f'X_prop_{prop_idx}'] = df[base_features[:7]].quantile(0.25, axis=1)
        prop_idx += 1
        df[f'X_prop_{prop_idx}'] = df[base_features[:7]].quantile(0.75, axis=1)
        prop_idx += 1
        df[f'X_prop_{prop_idx}'] = (df[base_features[:5]] > df[base_features[:5]].mean(axis=1).values[:, None]).sum(axis=1)
        prop_idx += 1
        df[f'X_prop_{prop_idx}'] = df[base_features[:5]].idxmax(axis=1).str.extract('(\d+)')[0].astype(float)
        prop_idx += 1
    
    # 2. Non-linear transformations (7 features)
    for i in range(7):
        feat = base_features[i % len(base_features)]
        if i < 2:
            df[f'X_prop_{prop_idx}'] = np.sign(df[feat]) * np.sqrt(np.abs(df[feat]))
        elif i < 4:
            df[f'X_prop_{prop_idx}'] = np.tanh(df[feat] / df[feat].std())
        elif i < 6:
            df[f'X_prop_{prop_idx}'] = 1 / (1 + np.exp(-df[feat] / df[feat].std()))  # Sigmoid
        else:
            df[f'X_prop_{prop_idx}'] = rankdata(df[feat]) / len(df)  # Rank transform
        prop_idx += 1
    
    # 3. Market interaction features (10 features)
    if 'volume' in df.columns:
        for i in range(3):
            df[f'X_prop_{prop_idx}'] = df[base_features[i]] * np.log1p(df['volume'])
            prop_idx += 1
    
    if 'order_flow_imbalance' in df.columns:
        for i in range(3):
            df[f'X_prop_{prop_idx}'] = df[base_features[i+3]] * df['order_flow_imbalance']
            prop_idx += 1
    
    if 'kyle_lambda' in df.columns:
        for i in range(2):
            df[f'X_prop_{prop_idx}'] = df[base_features[i+6]] * np.sign(df['kyle_lambda']) * np.log1p(np.abs(df['kyle_lambda']))
            prop_idx += 1
    
    if 'vpin' in df.columns:
        for i in range(2):
            df[f'X_prop_{prop_idx}'] = df[base_features[i+8]] * df['vpin']
            prop_idx += 1
    
    # 4. Interaction ratios (5 features)
    for i in range(5):
        feat1 = base_features[i % len(base_features)]
        feat2 = base_features[(i + 1) % len(base_features)]
        df[f'X_prop_{prop_idx}'] = df[feat1] / (np.abs(df[feat2]) + 1e-8)
        prop_idx += 1
    
    # Handle infinities and NaN
    df = df.replace([np.inf, -np.inf], np.nan)
    df = df.fillna(0)
    
    print(f"Created {prop_idx-1} proprietary X variables")
    return df

def create_interaction_features(df, selected_features, n_interactions=50):
    """Create high-quality interaction features"""
    print(f"Creating {n_interactions} interaction features...")
    
    interaction_features = []
    feature_names = []
    
    # Prioritize features
    important_x = ["X752", "X287", "X298", "X759", "X302", "X55", "X56", "X52", "X303", "X51"]
    market_features = ['order_flow_imbalance', 'kyle_lambda', 'vpin', 'liquidity_imbalance',
                      'bid_ask_spread', 'buying_pressure', 'volume', 'log_volume']
    
    # Get available priority features
    priority_x = [f for f in important_x if f in selected_features and f in df.columns][:10]
    priority_market = [f for f in market_features if f in selected_features and f in df.columns][:8]
    
    interaction_count = 0
    
    # 1. X features with market microstructure (20 interactions)
    for i, x_feat in enumerate(priority_x[:10]):
        if interaction_count >= 20:
            break
        for j, market_feat in enumerate(priority_market[:4]):
            if interaction_count >= 20:
                break
            
            # Multiplication
            interaction_features.append(df[x_feat] * df[market_feat])
            feature_names.append(f'{x_feat}_x_{market_feat}')
            interaction_count += 1
            
            # Log interaction
            if interaction_count < 20 and market_feat in ['volume', 'kyle_lambda']:
                interaction_features.append(df[x_feat] * np.log1p(np.abs(df[market_feat])))
                feature_names.append(f'{x_feat}_x_log_{market_feat}')
                interaction_count += 1
    
    # 2. Market feature interactions (15 interactions)
    market_pairs = [
        ('order_flow_imbalance', 'kyle_lambda'),
        ('vpin', 'liquidity_imbalance'),
        ('buying_pressure', 'selling_pressure'),
        ('bid_ask_spread', 'total_liquidity'),
        ('volume', 'liquidity_ratio')
    ]
    
    for feat1, feat2 in market_pairs:
        if interaction_count >= 35:
            break
        if feat1 in df.columns and feat2 in df.columns:
            # Product
            interaction_features.append(df[feat1] * df[feat2])
            feature_names.append(f'{feat1}_x_{feat2}')
            interaction_count += 1
            
            # Ratio
            if interaction_count < 35:
                interaction_features.append(df[feat1] / (np.abs(df[feat2]) + 1e-8))
                feature_names.append(f'{feat1}_div_{feat2}')
                interaction_count += 1
            
            # Difference
            if interaction_count < 35:
                interaction_features.append(df[feat1] - df[feat2])
                feature_names.append(f'{feat1}_minus_{feat2}')
                interaction_count += 1
    
    # 3. Non-linear interactions (10 interactions)
    for i in range(5):
        if interaction_count >= 45:
            break
        if i < len(priority_x):
            feat = priority_x[i]
            
            # Squared
            interaction_features.append(df[feat] ** 2)
            feature_names.append(f'{feat}_squared')
            interaction_count += 1
            
            # Square root of absolute
            if interaction_count < 45:
                interaction_features.append(np.sqrt(np.abs(df[feat])))
                feature_names.append(f'{feat}_sqrt_abs')
                interaction_count += 1
    
    # 4. Three-way interactions (5 interactions)
    if len(priority_x) >= 3 and len(priority_market) >= 1:
        for i in range(5):
            if interaction_count >= n_interactions:
                break
            x1 = priority_x[i % len(priority_x)]
            x2 = priority_x[(i+1) % len(priority_x)]
            m1 = priority_market[i % len(priority_market)]
            
            interaction_features.append(df[x1] * df[x2] * df[m1])
            feature_names.append(f'{x1}_{x2}_{m1}')
            interaction_count += 1
    
    # Create DataFrame
    interaction_df = pd.DataFrame(
        np.column_stack(interaction_features[:interaction_count]),
        columns=feature_names[:interaction_count],
        index=df.index
    )
    
    # Handle infinities and NaN
    interaction_df = interaction_df.replace([np.inf, -np.inf], np.nan)
    interaction_df = interaction_df.fillna(0)
    
    print(f"Created {interaction_count} interaction features")
    return interaction_df

def add_features(df):
    """Create comprehensive features for market microstructure"""
    print("Engineering features...")
    
    # Basic interactions (from original)
    df['bid_ask_spread'] = df['ask_qty'] - df['bid_qty']
    df['bid_ask_ratio'] = df['bid_qty'] / (df['ask_qty'] + 1e-8)
    df['buy_sell_ratio'] = df['buy_qty'] / (df['sell_qty'] + 1e-8)
    df['order_flow_imbalance'] = (df['buy_qty'] - df['sell_qty']) / (df['volume'] + 1e-8)
    
    # Pressure indicators
    df['buying_pressure'] = df['buy_qty'] / (df['volume'] + 1e-8)
    df['selling_pressure'] = df['sell_qty'] / (df['volume'] + 1e-8)
    df['net_pressure'] = df['buying_pressure'] - df['selling_pressure']
    
    # Liquidity features
    df['total_liquidity'] = df['bid_qty'] + df['ask_qty']
    df['liquidity_imbalance'] = (df['bid_qty'] - df['ask_qty']) / (df['total_liquidity'] + 1e-8)
    df['liquidity_ratio'] = df['total_liquidity'] / (df['volume'] + 1e-8)
    
    # Volume transformations
    df['log_volume'] = np.log1p(df['volume'])
    df['sqrt_volume'] = np.sqrt(df['volume'])
    
    # Market microstructure
    df['kyle_lambda'] = df['order_flow_imbalance'] / (df['sqrt_volume'] + 1e-8)
    df['vpin'] = np.abs(df['buy_qty'] - df['sell_qty']) / (df['buy_qty'] + df['sell_qty'] + 1e-8)
    
    # Additional useful features from extended version
    df['effective_spread'] = 2 * np.abs(df['order_flow_imbalance']) * df['bid_ask_spread']
    df['realized_spread'] = df['bid_ask_spread'] * df['vpin']
    df['price_impact'] = df['kyle_lambda'] * df['volume']
    df['trade_intensity'] = df['volume'] / (df['total_liquidity'] + 1e-8)
    
    # Handle infinities and NaN
    df = df.replace([np.inf, -np.inf], np.nan)
    df = df.fillna(0)
    
    return df

# Main bootstrap pipeline
def run_bootstrap_iteration(train, test, iteration, tracker, cfg):
    """Run one bootstrap iteration"""
    print(f"\n{'='*60}")
    print(f"Bootstrap Iteration {iteration + 1}/{cfg.n_bootstraps}")
    print(f"{'='*60}")
    
    # Create bootstrap sample
    train_boot, boot_indices = create_bootstrap_sample(
        train, 
        sample_ratio=cfg.bootstrap_sample_ratio,
        random_state=cfg.random_state + iteration
    )
    tracker.bootstrap_indices.append(boot_indices)
    
    # Create proprietary features
    train_boot = create_proprietary_x_variables(train_boot, cfg.n_proprietary_features)
    test_boot = test.copy()
    test_boot = create_proprietary_x_variables(test_boot, cfg.n_proprietary_features)
    
    # Feature engineering
    train_boot = add_features(train_boot)
    test_boot = add_features(test_boot)
    
    # Select base features
    selected_x_features = [
        "X752", "X287", "X298", "X759", "X302", "X55", "X56", "X52", "X303", "X51",
        "X344", "X598", "X385", "X603", "X674", "X415", "X345", "X137", "X174", "X178"
    ]
    
    # Add proprietary features
    proprietary_features = [f"X_prop_{i}" for i in range(1, cfg.n_proprietary_features + 1)]
    selected_x_features.extend(proprietary_features)
    
    # Get all X features and add more if needed
    all_x_features = [col for col in train_boot.columns if col.startswith('X') and col[1:].isdigit()]
    additional_x = [f for f in all_x_features if f not in selected_x_features][:cfg.max_x_features - len(selected_x_features)]
    selected_x_features.extend(additional_x)
    
    available_x_features = [f for f in selected_x_features if f in train_boot.columns]
    
    # Market and engineered features
    market_features = ['bid_qty', 'ask_qty', 'buy_qty', 'sell_qty', 'volume']
    engineered_features = [col for col in train_boot.columns if col in [
        'bid_ask_spread', 'bid_ask_ratio', 'buy_sell_ratio', 'order_flow_imbalance',
        'buying_pressure', 'selling_pressure', 'net_pressure', 'total_liquidity', 
        'liquidity_imbalance', 'liquidity_ratio', 'log_volume', 'sqrt_volume',
        'kyle_lambda', 'vpin', 'effective_spread', 'realized_spread', 
        'price_impact', 'trade_intensity'
    ]]
    
    # Combine base features
    base_selected_features = market_features + available_x_features + engineered_features
    base_selected_features = list(dict.fromkeys(base_selected_features))
    base_selected_features = [f for f in base_selected_features if f in train_boot.columns]
    
    # Create interaction features
    interaction_df_train = create_interaction_features(train_boot, base_selected_features, cfg.n_interaction_features)
    interaction_df_test = create_interaction_features(test_boot, base_selected_features, cfg.n_interaction_features)
    
    # Add interaction features
    for col in interaction_df_train.columns:
        train_boot[col] = interaction_df_train[col]
        test_boot[col] = interaction_df_test[col]
    
    # All features before selection
    all_features = base_selected_features + list(interaction_df_train.columns)
    
    # Prepare data for feature selection
    X_train_all = train_boot[all_features]
    y_train = train_boot['label']
    X_test_all = test_boot[all_features]
    
    # Feature selection with stability
    if cfg.use_feature_selection:
        importance_df = get_stable_feature_importance(
            X_train_all, y_train, all_features, n_runs=3
        )
        
        # Select features based on mean importance and stability
        selected_features = importance_df[
            (importance_df['importance_mean'] > cfg.feature_selection_threshold) &
            (importance_df['importance_cv'] < 0.5)  # Coefficient of variation < 0.5
        ]['feature'].tolist()
        
        # Always include critical features
        critical_features = ['order_flow_imbalance', 'kyle_lambda', 'vpin', 'volume', 
                            'bid_ask_spread', 'liquidity_imbalance', 'buying_pressure']
        
        for feat in critical_features:
            if feat in all_features and feat not in selected_features:
                selected_features.append(feat)
    else:
        selected_features = all_features
        importance_df = pd.DataFrame()
    
    X_train = X_train_all[selected_features]
    X_test = X_test_all[selected_features]
    
    print(f"Selected {len(selected_features)} features for this iteration")
    
    # Initialize storage for this iteration
    cv_scores = {}
    train_scores = {}
    predictions = {}
    
    # Model configurations
    model_configs = [
        {
            'name': 'conservative',
            'params': {
                'n_estimators': 800,
                'max_depth': 5,
                'learning_rate': 0.012,
                'subsample': 0.65,
                'colsample_bytree': 0.65,
                'reg_alpha': 2.5,
                'reg_lambda': 2.5,
                'min_child_weight': 25,
                'gamma': 0.3,
                'random_state': 42,
                'n_jobs': -1,
                'verbosity': 0
            }
        },
        {
            'name': 'balanced',
            'params': {
                'n_estimators': 600,
                'max_depth': 7,
                'learning_rate': 0.018,
                'subsample': 0.75,
                'colsample_bytree': 0.75,
                'reg_alpha': 1.0,
                'reg_lambda': 1.0,
                'min_child_weight': 10,
                'gamma': 0.1,
                'random_state': 42,
                'n_jobs': -1,
                'verbosity': 0
            }
        },
        {
            'name': 'aggressive',
            'params': {
                'n_estimators': 500,
                'max_depth': 9,
                'learning_rate': 0.025,
                'subsample': 0.85,
                'colsample_bytree': 0.85,
                'reg_alpha': 0.3,
                'reg_lambda': 0.3,
                'min_child_weight': 5,
                'gamma': 0.05,
                'random_state': 42,
                'n_jobs': -1,
                'verbosity': 0
            }
        }
    ]
    
    # Standardize data
    scaler = StandardScaler()
    X_train_scaled = pd.DataFrame(
        scaler.fit_transform(X_train),
        columns=selected_features,
        index=X_train.index
    )
    X_test_scaled = pd.DataFrame(
        scaler.transform(X_test),
        columns=selected_features,
        index=X_test.index
    )
    
    # Train and evaluate each model configuration
    for config in model_configs:
        print(f"\nEvaluating {config['name']} model...")
        
        # Cross-validation
        cv_mean, cv_std, fold_scores = stable_cross_validation(
            X_train_scaled, y_train, config['params'], 
            n_folds=cfg.cv_folds, 
            random_state=cfg.random_state + iteration
        )
        
        # Full training
        model = XGBRegressor(**config['params'])
        model.fit(X_train_scaled, y_train)
        
        train_pred = model.predict(X_train_scaled)
        test_pred = model.predict(X_test_scaled)
        
        train_score = pearsonr(y_train, train_pred)[0]
        
        # Store results
        cv_scores[config['name']] = cv_mean
        train_scores[config['name']] = train_score
        predictions[config['name']] = test_pred
        
        print(f"  CV Score: {cv_mean:.4f} (±{cv_std:.4f})")
        print(f"  Train Score: {train_score:.4f}")
        print(f"  Overfitting: {train_score - cv_mean:.4f}")
    
    # Record iteration results
    tracker.record_iteration(
        iteration, selected_features, importance_df,
        cv_scores, train_scores, predictions
    )
    
    return selected_features, cv_scores, train_scores

# Main execution
print("\nLoading data...")
train = pd.read_parquet(CFG.train_path)
test = pd.read_parquet(CFG.test_path)
submission = pd.read_csv(CFG.sample_sub_path)

print(f"Train shape: {train.shape}")
print(f"Test shape: {test.shape}")

# Memory optimization
train = reduce_mem_usage(train, "train")
test = reduce_mem_usage(test, "test")

# Initialize tracker
tracker = BootstrapStabilityTracker()

# Run bootstrap iterations
for i in range(CFG.n_bootstraps):
    selected_features, cv_scores, train_scores = run_bootstrap_iteration(
        train, test, i, tracker, CFG
    )

# Analyze results
print("\n" + "="*80)
print("Bootstrap Stability Analysis Results")
print("="*80)

# Get stable features
stable_features = tracker.get_stable_features(CFG.feature_stability_threshold)
print(f"\nFound {len(stable_features)} stable features (appearing in ≥{CFG.feature_stability_threshold*100}% of bootstraps)")
print("\nTop 20 most stable features:")
for feat, freq in stable_features[:20]:
    print(f"  {feat}: {freq*100:.1f}%")

# Get model stability metrics
stability_metrics = tracker.get_model_stability_metrics()
print("\nModel Stability Metrics:")
print("-" * 60)
print(f"{'Model':<15} {'CV Mean':<10} {'CV Std':<10} {'Train Mean':<10} {'Overfit':<10}")
print("-" * 60)

for model_name, metrics in stability_metrics.items():
    print(f"{model_name:<15} {metrics['cv_mean']:<10.4f} {metrics['cv_std']:<10.4f} "
          f"{metrics['train_mean']:<10.4f} {metrics['overfit_score']:<10.4f}")

# Find most stable model
most_stable_model = min(stability_metrics.items(), 
                       key=lambda x: x[1]['cv_std'] + abs(x[1]['overfit_score']))
print(f"\nMost stable model: {most_stable_model[0]}")

# Create final ensemble using stable features only
print("\n" + "="*80)
print("Creating Final Stable Ensemble")
print("="*80)

# Use only features that appear in most bootstraps
final_features = [feat for feat, freq in stable_features if freq >= CFG.feature_stability_threshold]

# Prepare final dataset with stable features
# (Re-engineer features on full dataset)
train_final = train.copy()
test_final = test.copy()

train_final = create_proprietary_x_variables(train_final, CFG.n_proprietary_features)
test_final = create_proprietary_x_variables(test_final, CFG.n_proprietary_features)

train_final = add_features(train_final)
test_final = add_features(test_final)

# Get available stable features
available_stable_features = [f for f in final_features if f in train_final.columns]

# Create final predictions using ensemble of stable models
final_predictions = []

for model_name in tracker.model_predictions:
    if stability_metrics[model_name]['cv_std'] < CFG.score_stability_threshold:
        # Average predictions from all bootstrap iterations for stable models
        model_preds = np.mean(tracker.model_predictions[model_name], axis=0)
        final_predictions.append(model_preds)
        print(f"Including {model_name} in final ensemble")

# Create final ensemble prediction
if final_predictions:
    final_pred = np.mean(final_predictions, axis=0)
else:
    print("Warning: No stable models found, using all models")
    final_pred = np.mean([np.mean(preds, axis=0) 
                         for preds in tracker.model_predictions.values()], axis=0)

# Post-processing
y_train = train['label']
p1, p99 = np.percentile(y_train, [1, 99])
final_pred = np.clip(final_pred, p1, p99)

# Create submission
submission['prediction'] = final_pred
submission.to_csv('stable_submission.csv', index=False)

print("\n" + "="*80)
print("Stable submission saved to stable_submission.csv")
print(submission.head())

# Create stability report
stability_report = pd.DataFrame({
    'feature': [f[0] for f in stable_features[:50]],
    'stability_score': [f[1] for f in stable_features[:50]]
})
stability_report.to_csv('feature_stability_report.csv', index=False)

model_stability_df = pd.DataFrame(stability_metrics).T
model_stability_df.to_csv('model_stability_report.csv')

# Plot stability analysis
tracker.plot_stability_analysis()

print("\nStability analysis complete!")
print("Files generated:")
print("  - stable_submission.csv: Final predictions using stable features/models")
print("  - feature_stability_report.csv: Feature stability analysis")
print("  - model_stability_report.csv: Model stability metrics")
print("  - bootstrap_stability_analysis.png: Visualization of stability analysis")

print("\n" + "="*80)
print("Bootstrap Stability Pipeline Completed Successfully!")
print("="*80)